In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
auctions = pd.read_csv('./datos/auctions.csv')

# A continuación vamos a analizar los tipos de datos del archivo *'auctions'* para poder reducir el tamaño del mismo

Primero que nada, vamos a ver que columnas se tienen y que tipo de dato les asigna Pandas a cada una de ellas.

Podemos ver que el archivo cuenta con **19.571.319 filas** y **7 columnas**, y pesa más de un GigaByte.

In [3]:
auctions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19571319 entries, 0 to 19571318
Data columns (total 7 columns):
auction_type_id    float64
country            int64
date               object
device_id          int64
platform           int64
ref_type_id        int64
source_id          int64
dtypes: float64(1), int64(5), object(1)
memory usage: 1.0+ GB


Luego de haber obtenido un listado de las columnas, vamos a ver cuales de ellas contienen nulos. 

La única que tiene nulos es la columna ***'auction_type_id'***, y tiene una cantidad de 19.571.319. Como podemos ver en la salida anterior, la cantidad de nulos es igual a la cantidad de filas de nuestro archivo, por lo que podemos eliminar dicha columna ya que todos los campos son nulos.

In [4]:
auctions.isnull().sum()

auction_type_id    19571319
country                   0
date                      0
device_id                 0
platform                  0
ref_type_id               0
source_id                 0
dtype: int64

In [6]:
auctions = auctions.drop(['auction_type_id'], axis='columns')

In [53]:
auctions.sample(5)

,country,device_id,platform,ref_type_id,source_id,day,hour
18375202,6333597102633388268,751142140,1,1,5,Friday,17
12264436,6333597102633388268,1521269610,1,1,5,Saturday,7
1395371,6333597102633388268,2001005456,1,1,1,Friday,22
5649573,6333597102633388268,577362306,1,1,0,Monday,22
16372910,6333597102633388268,-2862200,1,1,1,Tuesday,5


Analizando la columna ***'country'*** podemos ver que para todos los datos se obtiene el mismo valor y está determinado por el tipo **int64**. Esto nos permite realizar dos acciones depende el enfoque del análisis que haremos con dicho archivo.
+ Si trabajamos solo con este archivo podemos desisir de dicha columna eliminándola para poder reducir el tamaño de los datos. Cuando lleguemos a una conclusión, sabremos de que se tratan de subastas solo para dicho país.
+ Ahora bien, si trabajamos junto con otro archivo, lo que podemos hacer es cambiarle el tipo de dato a ***'category'***. 

En ambas formas, reduciremos bastante el tamaño del archivo.

In [8]:
auctions['country'].value_counts()

6333597102633388268    19571319
Name: country, dtype: int64

In [12]:
auctions['country'] = auctions['country'].astype('category')

De una manera bastante parecida podemos tratar a algunas columnas, en ellas no tenemos un solo valor por lo que no es recomendable eliminarlas (salvo que nuestro análisis esté enfocado a otros datos), pero se tienen sólo una cantidad finita (y pequeña) de valores posibles. Por ende, nos servirá cambiar el tipo de **int64** a **category**.
Las columnas son las siguientes: 
+ ***'platform'***
+ ***'ref_type_id'***
+ ***'source_id'***

In [15]:
auctions['platform'].value_counts()

1    15541825
2     4029494
Name: platform, dtype: int64

In [23]:
auctions['platform'] = auctions['platform'].astype('category')

In [24]:
auctions['ref_type_id'].value_counts()

1    15541825
7     4029494
Name: ref_type_id, dtype: int64

In [28]:
auctions['ref_type_id'] = auctions['ref_type_id'].astype('category')

In [30]:
auctions['source_id'].value_counts()

0    13354597
1     4016739
5     1466494
2      582083
6      151406
Name: source_id, dtype: int64

In [31]:
auctions['source_id'] = auctions['source_id'].astype('category')

In [32]:
auctions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19571319 entries, 0 to 19571318
Data columns (total 6 columns):
country        category
date           object
device_id      int64
platform       category
ref_type_id    category
source_id      category
dtypes: category(4), int64(1), object(1)
memory usage: 373.3+ MB


Si nos centramos en la columna ***'date'***, podemos dividir en cuatro columnas más, para poder así separar los datos y entenderlos mejor. Primero lo que debemos hacer es transformar dicha columna al tipo **datetime**.

Después la separaremos en ***'year'***, ***'month'***, ***'day'*** y ***'hour'***.

In [33]:
auctions['date'] = pd.to_datetime(auctions['date'])

In [39]:
auctions['year'] = auctions['date'].dt.year
auctions['month'] = auctions['date'].dt.month
auctions['day'] = auctions['date'].dt.day
auctions['hour'] = auctions['date'].dt.hour

C:\Users\maxi_\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: `weekday_name` is deprecated and will be removed in a future version. Use `day_name` instead
  This is separate from the ipykernel package so we can avoid doing imports until


KeyboardInterrupt: 

In [36]:
auctions.sample(5)

,country,date,device_id,platform,ref_type_id,source_id,year,month,day,hour
18901904,6333597102633388268,2019-03-13 03:57:22.878695,2531009007975425268,1,1,0,2019,3,13,3
1796506,6333597102633388268,2019-03-11 20:11:55.096463,9007977094343713028,1,1,2,2019,3,11,20
3590439,6333597102633388268,2019-03-08 20:15:44.749790,1627039692148031491,1,1,0,2019,3,8,20
3817230,6333597102633388268,2019-03-10 15:52:15.475765,2603919716887070282,1,1,0,2019,3,10,15
18072359,6333597102633388268,2019-03-13 19:54:59.745085,140129128652332675,1,1,0,2019,3,13,19


Si analizamos las columnas ***'year'*** y ***'month'*** podemos ver que cada una solo tiene un valor posible, es decir, que los datos representados en nuetro archivo corresponden al mes de Marzo de 2019. Por lo que podemos eliminar dichas columnas, teniendo en cuenta al final, que el análisis corresponden a dicho período de tiempo.

In [37]:
auctions['year'].value_counts()

2019    19571319
Name: year, dtype: int64

In [38]:
auctions['month'].value_counts()

3    19571319
Name: month, dtype: int64

In [40]:
auctions = auctions.drop(['year'], axis='columns')
auctions = auctions.drop(['month'], axis='columns')
auctions = auctions.drop(['date'], axis='columns')

In [41]:
auctions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19571319 entries, 0 to 19571318
Data columns (total 7 columns):
country        category
device_id      int64
platform       category
ref_type_id    category
source_id      category
day            object
hour           int64
dtypes: category(4), int64(2), object(1)
memory usage: 522.6+ MB


Como claramente la hora de las subastas es un número entre 0 y 23, podemos tratar a la columna ***'hour'*** como un **int8** ya que no es necesario representarla a través de un **int64**.
    

In [42]:
auctions['hour'] = auctions['hour'].astype(np.int8)

In [46]:
auctions['day'].value_counts()

Wednesday    3971343
Tuesday      3962311
Sunday       2637534
Monday       2574916
Friday       2303002
Saturday     2074552
Thursday     2047661
Name: day, dtype: int64

In [49]:
auctions['day'] = auctions['day'].astype('category')

In [50]:
auctions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19571319 entries, 0 to 19571318
Data columns (total 7 columns):
country        category
device_id      int64
platform       category
ref_type_id    category
source_id      category
day            category
hour           int8
dtypes: category(5), int64(1), int8(1)
memory usage: 261.3 MB
